In [ ]:
from dotenv import load_dotenv
load_dotenv()

## Chat Models

In [ ]:
# Method 1
from langchain_groq import ChatGroq

model = ChatGroq(model="openai/gpt-oss-120b", temperature=0)
model

In [ ]:
# Meth 2
from langchain.chat_models import init_chat_model #https://js.langchain.com/docs/integrations/chat/

LLM = init_chat_model(model="groq:openai/gpt-oss-120b")
LLM

In [ ]:
LLM.invoke("Hi!")

## Prompts
A prompt is the text we give to the LLM (or ChatModel). <br>
LangChain provides special classes like PromptTemplate and ChatPromptTemplate to make prompts dynamic and reusable.

In [ ]:
# Single Input Variable
from langchain_core.prompts import PromptTemplate

template = PromptTemplate(
    template= "Explain {topic} in simple Term"
)

prompt = template.format(topic = "Large Language Language")
prompt

In [ ]:
response = model.invoke(prompt)
response.content

In [ ]:
# ChatPromptTemplate 
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are helpfull assistent"),
    ("user", "Summarize the concept of {concept} in 3 bullet points.")
])

message = template.format(concept = "Large Language Model")
message

In [ ]:
response = model.invoke(message)
response.content

In [ ]:
# Using Message Types
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage

template = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a wise and slightly humorous AI mentor."),
    HumanMessage(content="Can you explain LangChain in one sentence?"),
    AIMessage(content="LangChain is a Python framework that helps you build apps powered by large language models."),
    HumanMessage(content="Great! Can you also give me one real-world use case?")
])

message = template.format_messages()
message

In [ ]:
model.invoke(message).content

## Chain in LangChain
A Chain is basically a pipeline that connects multiple steps together, where the output of one step becomes the input of the next.
https://python.langchain.com/api_reference/langchain/chains.html

### 1. Single-Step Chain (LLMChain)
This is the simplest form — Prompt → LLM → Output.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} in simple terms and add one fun fact."
)

chain = LLMChain(llm=model, prompt=prompt)

result = chain.invoke("Quantum Computing")
result

In [ ]:
# Runnable chain
# Use pipe | operator to make a chain
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
res = chain.invoke("Cloud Computing")
res

### 2. Multi-Step Chain (SimpleSequentialChain)
This combines two LLM steps, first create an outline, then expand it into a blog post.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain_core.output_parsers import StrOutputParser

outline_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Create a 3-point outline for a blog post about {topic}."
)

outline_chain = LLMChain(llm=model, prompt=outline_prompt)

# Step 2: Content generator
content_prompt = PromptTemplate(
    input_variables=["outline"],
    template="Expand this outline into a detailed 200-word blog post:\n{outline}"
)
content_chain = LLMChain(llm=model, prompt=content_prompt)

# Combine into one chain
blog_chain = SimpleSequentialChain(chains=[outline_chain, content_chain])

# Run the full pipeline
final_result = blog_chain.invoke("The Future of Renewable Energy")
print(final_result)


In [ ]:
# Runnable chain
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain_core.output_parsers import StrOutputParser

outline_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Create a 3-point outline for a blog post about {topic}."
)

outline_chain = outline_prompt | model | StrOutputParser()

content_prompt = PromptTemplate(
    input_variables=["outline"],
    template="Expand this outline into a detailed 200-word blog post:\n{outline}"
)
content_chain = content_prompt | model | StrOutputParser()

blog_chain = outline_chain | content_chain | model | StrOutputParser()

# Run the full pipeline
final_result = blog_chain.invoke({"The Future of Renewable Energy"})
print(final_result)

##  Memory in LangChain
By default, LangChain calls are stateless, the model only sees the current input and forgets everything else.
https://python.langchain.com/api_reference/langchain/memory.html

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

def withoutMemory():
    # Prompt template
    prompt = PromptTemplate(
        input_variables=["question"],
        template="""You are a friendly chatbot.
User: {question}
Assistant:
"""
    )

    # Create chain
    chain = LLMChain(llm=model, prompt=prompt)

    print("🤖 Chatbot is ready! Type 'exit' to stop.\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            break
        response = chain.run(user_input)
        print(f"Bot: {response}")

In [ ]:
withoutMemory()

### ConversationBufferMemory
- Stores: Full conversation history.
- Use: Small chats where cost & token limits aren’t an issue.
- Pro: Simple and easy.
- Con: Can grow too large for long sessions.

In [ ]:
system_prompts = """You are an AI Assistant for **Advanced Telecom Services (ATS)**. Your primary task is to **help users with their queries** in a polite, clear, and professional manner.
Conversation so far:
{chat_history}

**About ATS:**
**Founded:** 2011, Washington, USA
**Focus:** Digital wireless (3G, LTE, 5G), testing, test automation, software development, cloud, and artificial intelligence.
**Mission:** Innovate and achieve operational efficiency for the telecom and wireless industry.
**Team:** Top minds delivering high‑quality engineering and technology services, partnering with customers for superior service delivery and success.
**Contact Information:**
**Website:** https://www.atsailab.com/
**Address:** 744 241st Ln SE, Sammamish, WA 98074.
**Email:** info@atsailab.com
**Phone:** +1 (425) 533-1351
**LinkedIn:** https://www.linkedin.com/company/atsailab/
**Your Role as an assistant:** Respond politely and helpfully to every user query.
Only respond with the information provided in this message. Do not hallucinate information. 
Provide accurate information about ATS's services, expertise, and contact details.
Offer guidance, answer technical questions, and assist with any related requests while maintaining a courteous tone.'

User: {question}
Assistant:
"""

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

def langchainMemory():
    
    # Create Memory
    memory = ConversationBufferMemory(memory_key="chat_history")

    # Prompt template
    prompt = PromptTemplate(
        input_variables=["chat_history", "question"],
        template=system_prompts
    )

    # Create chain
    chain = LLMChain(llm=model, prompt=prompt, memory=memory)

    print("🤖 Chatbot is ready! Type 'exit' to stop.\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            break
        response = chain.run(user_input)
        print(f"Bot: {response}")

In [ ]:
langchainMemory()

### ConversationBufferWindowMemory
- Stores: Only the last N interactions (k).
- Use: Long-running chats, avoid sending huge history.
- Pro: Efficient, keeps relevant context.
- Con: Loses older history.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferWindowMemory

def interactive_chat_with_window_memory(window_size=3):

    # Create windowed memory
    memory = ConversationBufferWindowMemory(
        memory_key="chat_history",
        k=window_size  # Only remember the last `k` exchanges
    )

    # Prompt template
    prompt = PromptTemplate(
        input_variables=["chat_history", "question"],
        template="""
You are a helpful chatbot that remembers only the last {window_size} exchanges.
Conversation so far:
{chat_history}

User: {question}
Assistant:
""".replace("{window_size}", str(window_size))
    )

    # Create chain
    chain = LLMChain(llm=model, prompt=prompt, memory=memory)

    print(f"🤖 Chatbot ready! Remembering last {window_size} exchanges. Type 'exit' to stop.\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            break
        response = chain.run(user_input)
        print(f"Bot: {response}")

In [ ]:
interactive_chat_with_window_memory(window_size=3)

### LangGraph Memory Saver
Production-friendly way to store chat state.

In [ ]:
system_prompts = """You are an AI Assistant for **Advanced Telecom Services (ATS)**. Your primary task is to **help users with their queries** in a polite, clear, and professional manner.
**About ATS:**
**Founded:** 2011, Washington, USA
**Focus:** Digital wireless (3G, LTE, 5G), testing, test automation, software development, cloud, and artificial intelligence.
**Mission:** Innovate and achieve operational efficiency for the telecom and wireless industry.
**Team:** Top minds delivering high‑quality engineering and technology services, partnering with customers for superior service delivery and success.
**Contact Information:**
**Website:** https://www.atsailab.com/
**Address:** 744 241st Ln SE, Sammamish, WA 98074.
**Email:** info@atsailab.com
**Phone:** +1 (425) 533-1351
**LinkedIn:** https://www.linkedin.com/company/atsailab/
**Your Role as an assistant:** Respond politely and helpfully to every user query.
Only respond with the information provided in this message. Do not hallucinate information. 
Provide accurate information about ATS's services, expertise, and contact details.
Offer guidance, answer technical questions, and assist with any related requests while maintaining a courteous tone.'
"""

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph
from langchain_core.messages import HumanMessage, SystemMessage

# Memory
memory = MemorySaver()

def chatbot_node(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": state["messages"] + [response]}

graph = StateGraph(MessagesState)
graph.add_node("chatbot", chatbot_node)
graph.set_entry_point("chatbot")

# Compile with memory
app = graph.compile(checkpointer=memory)
# app

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    # Invoke graph with the session ID (for persistence)
    result = app.invoke({"messages": [system_prompts, HumanMessage(content=user_input)]},
                        config={"configurable": {"thread_id": "1"}})

    # Get last assistant message
    result["messages"][-1].pretty_print()